In [1]:
import caliper, caliperpy, traceback, sys
import os
import sys
path = "c:/Users/JacobFo/OneDrive - City of Durham/Python/TRMG2/TRMG2_Analyses"
os.chdir(path)


import caliper, caliperpy, traceback, sys
import numpy as np
import pandas as pd
from siuba import *
from siuba import group_by, summarize, _
from plotnine import ggplot, aes, geom_line
import voila
from plotnine import *




pd.set_option('display.float_format', lambda x: '%.3f' % x)
#git instructions - open terminal in jupyternotebook
## cd "C:/Users/JacobFo/OneDrive - City of Durham/Python/TRMG2"
dk = caliperpy.TransCAD.connect()

Connecting to TransCAD...


In [2]:
##TRMG2

path = "c:/Users/JacobFo/TRMG2_v0.2/scenarios/base_2016/output/networks/scenario_links.bin"
hwy_2016_g2 = dk.GetDataFrameFromBin(path)

hwy_2016_g2['Total_Transit_Flow'] = hwy_2016_g2['AB_Total_TransitFlow'] + hwy_2016_g2['BA_Total_TransitFlow']

##Load TRMv6.2

path = "c:/Users/JacobFo/OneDrive - City of Durham/Full Model Runs/Original TRMv6.2_2016 - Copy/"
inpFname =  os.path.join(path, "Input/Highway/Highway_Line.bin")
hwy_2016_v6 = dk.GetDataFrameFromBin(inpFname)

replace_nas = hwy_2016_v6

replace_nas.fillna(value = 0,inplace = True)

replace_nas['TotVMT'] = replace_nas['ABAMVMT']+replace_nas['ABMDVMT']+replace_nas['ABNTVMT']+replace_nas['ABPMVMT']+replace_nas['BAAMVMT']+replace_nas['BAMDVMT']+replace_nas['BANTVMT']+replace_nas['BAPMVMT']

replace_nas['TotVHT'] = replace_nas['ABAMVHT']+replace_nas['ABMDVHT']+replace_nas['ABNTVHT']+replace_nas['ABPMVHT']+replace_nas['BAAMVHT']+replace_nas['BAMDVHT']+replace_nas['BANTVHT']+replace_nas['BAPMVHT']

path = "c:/Users/JacobFo/OneDrive - City of Durham/Full Model Runs/Original TRMv6.2_2016 - Copy/"
inpFname =  os.path.join(path, "EvalModule/TRM6_Count_2016_Observed.bin")
count = dk.GetDataFrameFromBin(inpFname)

temp = (
count 
    >>select(_.ID, _.Daily_Counts)
)

temp = pd.merge(replace_nas, temp)

hwy_2016_v6_counts = (
temp
    >>filter(_.Daily_Counts > 0)
)



pd.set_option('display.float_format', lambda x: '%.3f' % x)

## How to use this tool:
aggregate_by_geo is basically a pivot + summarize function for any category in the TRMG2, making it easy to quickly summarize model outputs

example: VMT statistics by County:


In [3]:
group_cols = ["MPO"]
sum_cols = ["Total_Flow_Daily"]
mean_cols = ["Total_Flow_Daily"]



by_county = geography_utils.aggregate_by_geography(
    hwy_2016_g2,
    group_cols=group_cols,
    sum_cols = sum_cols,
    mean_cols=mean_cols,


)

by_county =   by_county.rename(columns={'Total_Flow_Daily_x': 'Sum Total Flow'})
by_county = by_county.rename(columns={'Total_Flow_Daily_y': 'Mean Flow'})
by_county

,MPO,Sum Total Flow,Mean Flow
0,CAMPO,194180089.684,2542.356
1,None,29152304.511,1274.529
2,DCHC,82426605.274,2930.828
3,None,NaN,NaN


In [4]:
group_cols = ["County"]
sum_cols = ["Total_Transit_Flow","Total_Flow_Daily"]


by_class = geography_utils.aggregate_by_geography(
  # replace_nas[replace_nas.COUNTY.notna()], 
    hwy_2016_g2,
    group_cols=group_cols,
    sum_cols = sum_cols


)

by_class

,County,Total_Flow_Daily,Total_Transit_Flow
0,Wake,172314595.946,2291254.297
1,Franklin,8323252.440,816.044
2,Johnston,19573688.226,3795.042
3,Chatham,5943315.381,21861.593
4,Durham,60486993.379,1753258.226
5,Nash,388466.360,41.788
6,Harnett,4795670.687,335.557
7,Orange,22718823.814,1142658.629
8,Alamance,5365310.614,8295.431
9,Granville,3324355.446,1282.284


In [5]:
import plotly
import plotly.express as px
import voila

In [6]:
import plotly.express as px

fig = px.bar(by_class,              # our dataframe
       x="County",         # x will be the 'key' column of the dataframe
       y="Total_Flow_Daily")

fig.show()